LSTM + Pytorch For User Stories Classification

1.1 Preprocess the data

In [1]:
from nltk.corpus import stopwords
from nltk import word_tokenize
import torch


#read dataset
import pandas as pd
df = pd.read_csv('training_data/NLI/train.csv')
premise_data = df['premise'].tolist()
hypothesis_data = df['hypothesis'].tolist()
label_data = df['label'].tolist()

In [5]:
import re

# data clean
paired_data = list(zip(premise_data, hypothesis_data, label_data))

duplicates = set()
unique_paired_data = []
for pair in paired_data:
    if (pair in duplicates) or (pair[0] == pair[1]):
        continue
    else:
        duplicates.add(pair)
        unique_paired_data.append(pair)

premise_data, hypothesis_data, label_data = zip(*unique_paired_data)
    
premise_data_clean_garbled = [re.sub(r'[^a-zA-Z0-9\s]', '', text) for text in premise_data]
hypothesis_data_clean_garbled = [re.sub(r'[^a-zA-Z0-9\s]', '', text) for text in hypothesis_data]

cleaned_premise_data = [' '.join(re.sub(r'\b\w*www\w*\b', '', text).split()) for text in premise_data_clean_garbled]
cleaned_hypothesis_data = [' '.join(re.sub(r'\b\w*www\w*\b', '', text).split()) for text in hypothesis_data_clean_garbled]

In [6]:
import nltk
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

preprocessed_premise_data = [word_tokenize(text) for text in cleaned_premise_data]
filtered_premise_data = [[word.lower() for word in premise if word.lower() not in stop_words] for premise in preprocessed_premise_data]


In [7]:
premise_data[274]

"The Furushoin pavilion's verandah is the perfect place for viewing the full moon."

1.2 Training : LSTM

In [4]:
# from sklearn.model_selection import train_test_split
# import torch
# import torch.nn as nn  
# import torch.optim as optim
# from torch.utils.data import TensorDataset, DataLoader
# # label data set
# labels = {'Usability':0,
#           'Functional':1,
#           'Maintainability':2,
#           'Security':3,
#           'Portability':4,
#           'Performance':5,
#           'Compatibility':6,
#           'Reliability':7
#           }
# labels_data = [labels[label] for label in label_data]


# class BERT_LSTM_Classifier(nn.Module):
#     def __init__(self, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional=True, dropout=0.5):
#         super(BERT_LSTM_Classifier, self).__init__()
#         self.lstm = nn.LSTM(input_size=embedding_dim,
#                             hidden_size=hidden_dim,
#                             num_layers=num_layers,
#                             bidirectional=bidirectional,
#                             batch_first=True,
#                             dropout=dropout if num_layers > 1 else 0)
#         self.directions = 2 if bidirectional else 1
#         self.fc = nn.Linear(hidden_dim * self.directions, output_dim)
#         self.log_softmax = nn.LogSoftmax(dim=1)

#     def forward(self, inputs):


#         lstm_out, _ = self.lstm(inputs)

   
#         out = self.fc(lstm_out)
    
#         out = self.log_softmax(out)
#         return out





# # load dataset and label
# from torch.utils.data import TensorDataset, DataLoader
# sentence_embeddings_tensor = torch.stack(sentence_embeddings)
# labels_tensor = torch.tensor(labels_data)

# X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings_tensor, labels_tensor, test_size=0.2, random_state=42)
# train_dataset = TensorDataset(X_train, y_train)
# test_dataset = TensorDataset(X_test, y_test)

# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# embedding_dim = 768
# hidden_dim = 256
# output_dim = len(labels)  
# num_epochs = 400 




# model = BERT_LSTM_Classifier(embedding_dim=embedding_dim,
#                              hidden_dim=hidden_dim,
#                              output_dim=output_dim,
#                              num_layers=1,  
#                              bidirectional=False,  
#                              dropout=0.1).to(device)

# #loss function
# loss_fn = nn.NLLLoss()  
# optimizer = optim.Adam(model.parameters())


# #train
# model.train()
# epoch_losses = []
# for epoch in range(num_epochs):
#     total_loss = 0
#     for inputs, labels in train_dataloader:
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_fn(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     avg_loss = total_loss / len(train_dataloader)
#     epoch_losses.append(avg_loss)
#     print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")


# model.eval()
# total = 0
# correct = 0
# with torch.no_grad():
#     for inputs, labels in test_dataloader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print(f"Accuracy on test set: {correct / total * 100:.2f}%")

# import matplotlib.pyplot as plt


# plt.figure(figsize=(10, 6))
# plt.plot(range(1, num_epochs+1), epoch_losses, marker='o', label='Training Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training Loss')
# plt.legend()
# plt.grid(True)
# plt.show()

Finetuning Bert Model